# Sistemas Inteligentes 2024/2025

## Projeto 2: Heurísticas para Pares de Pinguins

<center> <img src="penguins.png" width="600" /> </center>

## Entrega e Submissão

### Entrega: 1 de Abril às 23:59

### Formação de grupos

A formação de grupos para o Projeto 2 está aberta até dia **25/março às 23h59**.

---
## <span style="color:red">**ATENÇÃO:**</span>

**TODOS os alunos devem formar grupo**, mesmo que seja um grupo com apenas uma pessoa para fazer o projeto individualmente. 

**Devem formar grupo NOVAMENTE**, mesmo que seja o mesmo grupo que no(s) projeto(s) anterior(es).

Quem não formar grupo antes do final do prazo terá **10% de PENALIZAÇÃO** na nota do projeto. Quem fizer a submissão final do código com a identificação errada do seu grupo terá também a mesma penalização.

---

### Quizz

Cada grupo deve completar a implementação das duas funções heurísticas: `Npairings` e `highestPairings` da classe `PenguinsPairs` e testá-la no link do quizz **Projeto 2** que está na página da disciplina, introduzindo aí o vosso código. Os vários elementos do grupo podem desenvolver, submeter e avaliar o código várias vezes, sendo a submissão com melhor nota a que será considerada.

Esse quizz é constituído por uma única pergunta que cobre a implementação dos dois métodos e que serão avaliados através de um conjunto de testes automáticos visíveis e outros invisíveis, valendo um total de 1.5 valores da avaliação da Unidade Curricular.

Os testes visíveis valem 6 em 20, enquanto que os testes invisíveis valem 14 em 20. A primeira heurística (`Npairings`) vale 9 em 20 e a segunda heurística (`highestPairings`) vale 11 em 20.

### Ficheiro Pyhton

Simultaneamente, é necessário submeter o script Python que contém todo o código submetido no quizz. **Só se pretende UMA submissão por grupo**. Esse ficheiro deve chamar-se **heuristicasPinguins_SInt_24_25_grupoXX.py**, em que substituem XX pelo número do grupo.

---

---

## Introdução

Considere o jogo dos pares de pinguins que estão numa ilha de gelo, que pode ou não ser quadrada (ver imagens com exemplos). A ilha é escorregadia e quando um pinguim se desloca terá de embater noutro pinguim ou num icebergue/obstáculo para parar, caso contrário irá cair na água e desaparecer. Os pinguins só se podem deslocar para Norte (N), Sul (S), Este (E) ou Oeste (O). Podem existir vários pinguins numa ilha e o objetivo é emparelhá-los todos no menor número de movimentos. 

## Objetivo

Dada a implementação em Python da modelização do jogo dos pares de pinguins como um problema de procura num grafo, de acordo com o módulo `searchPlus.py`, deve completá-la implementantdo duas funções heurísticas que estimam o menor custo até ao objetivo: as funções **Npairings** and **highestPairings**.

## Formulação em Python

De seguida apresenta-se a implementação da classe `PenguinsPairs` para a modelização do problema, como um problema de procura num grafo, de acordo com o Paradigma do Espaço de Estados, que foi usada como referência para a realização dos testes do Projeto 1.

Considere que cada célula do mundo é dada por um par de coordenadas $(l,c)$ e que cada célula à esquerda de topo é a célula $(0,0)$ e que a célula do fundo à direita é $(M-1,N-1)$, com $M$ o número de linhas e $N$ o número de colunas do mundo.

Relembre que na representação do mundo considera-se que:
* a ilha de gelo é cercada por icebergues ou água, onde os icebergues são representados pelo símbolo "##" e a água por "()";
* o interior da ilha também pode conter icebergues ou buracos de água (representados pelos mesmos símbolos);
* existem 2 ou mais pinguins (até um máximo de 100) representados por dois caracteres numéricos, "00", "01", e por aí adiante;
* as casas livres são representadas por ".."

Nesta modelização, o estado contém a informação mínima. Apenas a informação que muda com as ações é incluída no estado, ou seja, a posição dos pinguins. Para tal, criámos uma subclasse do `namedtuple` para guardar a posição dos pinguins
* `pinguins` - dicionário em que a chave é o ID do pinguim e o valor as coordenadas da posição do pinguim

Para a restante informação do mundo (informação estática, que não muda com as ações), criámos os seguintes atributos:
* `dim` - a dimensão do mapa $(M,N)$
* `obstacles` - conjunto com as coordenadas dos icebergues
* `water` - conjunto com as coordenadas da água

In [ ]:
from searchPlus import *
from collections import namedtuple


line1 = "## ## ## ## ## ## ## ## ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## .. .. .. 00 .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. .. ##\n"
line7 = "## .. .. .. 01 .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## ## ## ## ## ## ## ## ##\n"
grid = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9


EstadoPin = namedtuple('EstadoPin','pinguins')

class EstadoPinguins(EstadoPin):
        
    def __hash__(self):
        return hash(tuple(sorted(self.pinguins.items())))
    
    def __lt__(self,other):
        return True
    
    
class PenguinsPairs(Problem):
    
    def process_txt(self, grid):
        """
        Função que porcessa a grelha em txt e obtém as posições dos icebergues (walls), água e pinguins.
        O output desta função é um dicionário em que as chaves identifica cada tipo de informação a ser 
        guardada. Os pinguins serão um dicionário e os icebergues e água serão um conjunto.
        """
        data = {'walls': set(), 'pinguins': {}, 'water': set()}
        lines = grid.split('\n')[:-1]
        x = 0
        for row in lines:
            ll = row.split()
            y = 0
            for col in ll:
                if col == '##':
                    data['walls'].add((x,y))
                elif col == '()':
                    data['water'].add((x,y))
                elif col != '..':
                    data['pinguins'][col] = (x,y)
                y += 1
            x += 1
        data['dim'] = (len(lines),len(ll))
        return data
    
    
    directions = {"N":(-1, 0), "E":(0, +1), "S":(+1, 0), "O":(0, -1)}  # ortogonais
    
    
    def __init__(self, ice_map=grid):
        initialStatus = self.process_txt(ice_map) # processa o txt e converte num dicionário
        self.initial = EstadoPinguins(initialStatus['pinguins']) # estado inicial
        self.goal = {} # estado vazio
        self.obstacles = initialStatus['walls'] # posições do icebergues
        self.water = initialStatus['water'] # posições da água
        self.dim = initialStatus['dim'] # dimensão do mapa (não precisa de ser quadrado)

    
    def slide(self,state,x,y,d):
        """
        Função que identifica se é possível o pinguim se deslocar numa direção.
        """
        (dx,dy) = self.directions[d]
        if (x+dx,y+dy) in self.obstacles:
            return False
        while (x+dx,y+dy) not in self.obstacles and (x+dx,y+dy) not in state.pinguins.values():
            if (x+dx,y+dy) in self.water:
                return False
            x = x+dx
            y = y+dy
        return True
    
    
    def actions (self, state):
        """
        Devolve uma lista ordenada com todas as ações possíveis para o estado.
        """
        actions_list = []
        for p in state.pinguins.keys():
            x,y = state.pinguins[p] # coordenadas da posição do pinguim 
            if self.slide(state,x,y,"N"): # verifica se o pinguim pode deslocar-se para Norte
                actions_list.append((p,"N"))
            if self.slide(state,x,y,"S"):
                actions_list.append((p,"S"))
            if self.slide(state,x,y,"E"):
                actions_list.append((p,"E"))
            if self.slide(state,x,y,"O"):
                actions_list.append((p,"O"))
        return sorted(actions_list) 
    

    def result (self, state, action):
        """
        Devolve um novo estado resultante de aplicar "action" a "state".
        """
        p,d = action # ação é um tuplo (ID pinguim, direção)
        pinguins = state.pinguins.copy()
        x,y = state.pinguins[p]
        (dx,dy) = self.directions[d]
        # desloca o pinguim até que embata num obstáculo ou noutro pinguim
        while (x+dx,y+dy) not in self.obstacles and (x+dx,y+dy) not in state.pinguins.values():
            x = x+dx
            y = y+dy
        # se o pinguim embateu num obstáculo, atualiza-se a posição do pinguim
        if (x+dx,y+dy) in self.obstacles:
            pinguins[p] = (x,y)
        # se o pinguim embateu noutro pinguim, ambos são removidos do estado
        if (x+dx,y+dy) in state.pinguins.values():
            pinguins.pop(p)
            for key in state.pinguins:
                if state.pinguins[key] == (x+dx,y+dy):
                    pinguins.pop(key)
                    break
        return EstadoPinguins(pinguins)
    

    def goal_test (self, state):
        """
        Verifica se todos os pinguins estão emparelhados, ou seja, se os estado está vazio.
        """
        return state.pinguins == {}
    

    def display (self, state):
        """
        Devolve a grelha em modo txt.
        """
        output = ""
        for i in range(self.dim[0]):
            for j in range(self.dim[1]):
                if (i,j) in state.pinguins.values():
                    for key in state.pinguins:
                        if state.pinguins[key] == (i,j):
                            ch = key
                            break
                elif (i,j) in self.obstacles:
                    ch = "##"
                elif (i,j) in self.water:
                    ch = "()"
                else:
                    ch = ".."
                output += ch + " "
            output += "\n"
        return output
    

    def executa(self, state, actions_list, verbose=False):
        """Executa uma sequência de acções a partir do estado devolvendo o triplo formado pelo estado, 
        pelo custo acumulado e pelo booleano que indica se o objectivo foi ou não atingido. Se o objectivo 
        for atingido antes da sequência ser atingida, devolve-se o estado e o custo corrente.
        Há o modo verboso e o não verboso, por defeito."""
        cost = 0
        for a in actions_list:
            seg = self.result(state,a)
            cost = self.path_cost(cost,state,a,seg)
            state = seg
            obj = self.goal_test(state)
            if verbose:
                print('Ação:', a)
                print(self.display(state),end='')
                print('Custo Total:',cost)
                print('Atingido o objectivo?', obj)
                print()
            if obj:
                break
        return (state, cost, obj)
    
    
    def halfPenguins(self, node):
        """
        Heurística que considera metade do número de pinguins.
        """
        return len(node.state.pinguins)//2
    
    
    def Npairings(self, node):
        pass
    
    
    def highestPairings(self, node):
        pass
    

Vamos confirmar que está a funcionar aplicando o algoritmo de procura largura-primeiro em grafo.

In [2]:
p = PenguinsPairs()
resultado = breadth_first_graph_search(p)
if resultado:
    print("Solução Larg-prim (grafo) com custo", str(resultado.path_cost)+":")
    print(resultado.solution())
else:
    print("Sem solução!")

Solução Larg-prim (grafo) com custo 3:
[('00', 'E'), ('01', 'E'), ('00', 'S')]


## Heurística 1: `halfPenguins`

O valor heurístico para qualquer estado corresponde a metade do número de pinguins. 

É fácil provar que esta **heurística é admissível** porque no melhor dos casos fazemos $n/2$ movimentos para emparelhar todos os pinguins (emparelhamentos diretos sem usar icebergues). Por outro lado, se tivermos de usar os icebergues teremos mais ações (custo superior) do que o valor heurístico (metade dos pinguins). 

Para qualquer ação (arco do grafo) $x \to y$ temos custo de 1. Para provarmos que a **heurística é consistente**, temos de analisar duas situações
* Quando deslocamos um pinguim contra um icebergue, mantemos o mesmo número de pinguins mas o custo da ação é 1. Logo, $h(x)-h(y)=n/2-n/2=0 \leq 1$
* Quando deslocamos um pinguim contra outro, passamos de $n/2$ pinguins num estado $x$ para $(n-2)/2$ pinguins no estado seguinte $y$. Logo, $h(x)-h(y)=n/2-(n-2)/2=1 \leq 1$

Vamos ver alguns exemplos desta heurística.

#### Exemplo 1:

No caso do mapa inicial fornecido, o valor da heurística é de 1. Vamos de seguida aplicar os algoritmos Greedy e A* em grafo para encontrar a solução e ver o número de estados expadidos para cada algoritmo.

In [3]:
p = PenguinsPairs()
print(p.halfPenguins(Node(p.initial)))

1


In [4]:
res,expanded = greedy_best_first_graph_search_plus_count(p,p.halfPenguins)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)

[('01', 'S'), ('01', 'O'), ('01', 'N'), ('01', 'E'), ('00', 'O'), ('01', 'S'), ('00', 'S'), ('00', 'E')]
Cost: 8
expanded: 8


In [5]:
res,expanded=astar_search_plus_count(p,p.halfPenguins)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)

[('01', 'E'), ('00', 'E'), ('00', 'S')]
Cost: 3
expanded: 15


#### Exemplo 2:

Consideremos agora um mapa diferente e com 8 pinguins. Vamos calcular o valor heurístico (que para o estado inicial será de 4) e aplicar novamente o Greedy e o A* em grafo para encontrar a solução.

In [6]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. 00 .. .. .. .. 03 ##\n"
line3 = "## .. .. 05 .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## 04 .. 06 01 .. .. .. ##\n"
line7 = "## .. .. .. 02 .. .. .. ##\n"
line8 = "## .. 07 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid2)
print(p.halfPenguins(Node(p.initial)))

4


In [7]:
res,expanded = greedy_best_first_graph_search_plus_count(p,p.halfPenguins)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)

[('04', 'E'), ('01', 'S'), ('00', 'S'), ('05', 'O'), ('03', 'O'), ('03', 'S')]
Cost: 6
expanded: 6


In [8]:
res,expanded=astar_search_plus_count(p,p.halfPenguins)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)

[('00', 'S'), ('05', 'E'), ('03', 'S'), ('04', 'E'), ('01', 'S')]
Cost: 5
expanded: 36


#### Exemplo 3:

Consideremos agora um exemplo com 10 pinguins. Neste caso, o valor heurístico será de 5. Vamos de seguida verificar o número de estado expandidos quando aplicamos o A* em grafo.

In [9]:
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid3)
print(p.halfPenguins(Node(p.initial)))

5


In [10]:
res,expanded=astar_search_plus_count(p,p.halfPenguins)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)

[('04', 'E'), ('03', 'E'), ('03', 'N'), ('09', 'E'), ('05', 'E'), ('05', 'S'), ('02', 'E'), ('01', 'O'), ('00', 'N')]
Cost: 9
expanded: 6007


## Heurística 2: `Npairings`

Esta heurística consiste em identificar o número de pares diretos que se conseguem formar somando os restantes pinguins que não se conseguem emparelhar:

$$Npairings = np + nsp$$

com $np$ o número de pares formados e $nsp$ o número de pinguins sem par.

O **emparelhamento direto de pinguins** corresponde a juntar pinguins que estão na **mesma linha ou coluna** e é feito respeitando a **ordem do ID dos pinguins**.

Vamos ver alguns exemplos desta heurística.

#### Exemplo 1:

No caso do mapa inicial fornecido o valor desta heurística é de 2 porque os pinguins não estão alinhados em linha ou coluna um com o outro, logo $np=0$ e sobram 2 pinguins sem par, ou seja, $nsp=2$.

```python
p = PenguinsPairs()
print(p.Npairings(Node(p.initial)))

out: 2
```

#### Exemplo 2:

Consideremos agora o mapa seguinte com 4 pinguins:
```
() () () () () () () () () 
## 02 00 .. .. .. .. 01 ## 
## .. .. .. .. .. .. .. ## 
## .. .. .. .. .. .. .. ## 
## .. .. () () () .. .. ## 
## .. .. .. .. .. .. 03 ## 
## .. .. .. .. .. .. .. ## 
## .. .. .. .. .. .. .. ## 
() () () () () () () () () 
```

Respeitando a ordem do ID, começamos pelo pinguim "00" e este pode-se emparelhar com dois pinguins que estão na mesma linha que ele (os pinguins "02" e o "01"). De acordo com a ordem do ID dos pinguins, o pinguim "00" deve-se emparelhar com o pinguim "01", e temos 1 par formado ($np=1$). Mas, agora os pinguins "02" e "03" não conseguem formar par diretamente, logo $nsp = 2$. Assim, $Npairings=3$.

```python
line1 = "() () () () () () () () ()\n"
line2 = "## 02 00 .. .. .. .. 01 ##\n"
line3 = "## .. .. .. .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. 03 ##\n"
line7 = "## .. .. .. .. .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid2)
print(p.Npairings(Node(p.initial)))

out: 3
```

#### Exemplo 3:

Vamos ver outro exemplo com mais pinguins, neste caso 10 pinguins:
```
() () () () () () () () () 
## .. .. 07 .. 01 .. .. ## 
## .. .. .. .. 05 .. .. ## 
## .. .. .. 09 .. .. .. ## 
## .. .. () () () 02 08 ## 
## .. .. .. .. 04 .. .. ## 
## .. 06 03 .. .. .. .. ## 
## .. 00 .. .. .. .. .. ## 
() () () () () () () () ()
```

A ordem de emparelhamento é a ordem dos IDs dos pinguins, ou seja, começamos por emparelhar "00", "01", e por aí adiante. Vamos ver quantos pares diretos conseguimos formar:
* "00" vai emparelhar com "06", temos 1 par formado
* "01" pode emparelhar com "07" e com "05", por ordem dos IDs, vai emparelhar com "05", temos outro par
* "02" vai emparelhar com "08", temos mais 1 par
* "03" poderia emparelhar com "06", mas "06" já foi emparelhado com "00"
* "04" não tem emparelhamento direto com nenhum pinguim
* "05" já foi emparelhado com "01"
* "06" já foi emparelhado com "00"
* "07" poderia emparelhar com "01", mas "01" foi emparelhado com "05"
* "08" foi emparelhado com "02"
* "09" não tem emparelhamento direto com nenhum pinguim

Ora, temos um total de 3 pares formados ("00" com "06", "01" com "05" e "02" com "08"), logo $np=3$ e sobram 4 pinguins sem emparelhamento, logo $nsp=4$.

```python
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid3)
print(p.Npairings(Node(p.initial)))

out: 7
```


Se aplicarmos o A* em grafo a este exemplo, teremos 673 estados expandidos (muito inferior ao número de estados expandidos pela heurística `halfPenguins`) e uma solução com custo 9.
```python
res,expanded=astar_search_plus_count(p,p.Npairings)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)
```
O código irá produzir o seguinte output:
```
[('04', 'E'), ('00', 'E'), ('09', 'O'), ('07', 'O'), ('07', 'S'), ('03', 'O'), ('02', 'E'), ('01', 'S'), ('00', 'N')]
Cost: 9
expanded: 673
```

## Heurística 3: `highestPairings`

Esta heurística consiste em identificar qual o pinguim com mais hipóteses de emparelhamento direto e este irá emparelhar com o pinguim que também terá mais hipóteses de emparelhamento direto. Depois de todos os emparelhamentos diretos efetuados, o valor heurístico é dado por 

$$highestPairings = np + nsp$$

com $np$ o número de pares formados e $nsp$ o número de pinguins sem par.

O **emparelhamento direto de pinguins** corresponde a juntar pinguins que estão na **mesma linha ou coluna** e é feito respeitando a **ordem dos pinguins com maior emparelhamento** e em caso de empate, usa-se o ID dos pinguins para desempatar.

Vamos ver alguns exemplos desta heurística.

#### Exemplo 1:

Consideremos o mapa seguinte com 4 pinguins:
```
() () () () () () () () () 
## 02 00 .. .. .. .. 01 ## 
## .. .. .. .. .. .. .. ## 
## .. .. .. .. .. .. .. ## 
## .. .. () () () .. .. ## 
## .. .. .. .. .. .. 03 ## 
## .. .. .. .. .. .. .. ## 
## .. .. .. .. .. .. .. ## 
() () () () () () () () () 
```

Vamos começar por identificar para cada pinguim os possíveis emparelhamentos.

| Pinguim | Emparelhamento direto |
| :------ | :-------------------- |
| 00      | 01 02                 |
| 01      | 00 03                 |
| 02      | 00                    |
| 03      | 01                    |


Respeitando a ordem de maiores emparelhamentos, temos os pinguins "00" e "01" empatados, então usaremos o ID destes pinguins para desempatar e começamos pelo pinguim "00". Este pode-se emparelhar com dois pinguins que estão na mesma linha que ele (os pinguins "02" e o "01"). De acordo com a ordem de maiores emparelhamentos (o pinguim "01" tem 2 emparelhamentos e o pinguim "02" tem 1 emparelhamento), o pinguim "00" deve-se emparelhar com o pinguim "01", e temos 1 par formado ($np=1$). Se continuarmos a percorrer a tabela pela ordem de maiores emparelhamentos (**não recalculamos novos emparelhamentos**), o seguinte que surge é o pinguim "01" que já foi emparelhado, e de seguida temos o pinguim "02" que só se poderia emparelhar com o pinguim "00", mas este último já foi emparelhado, deixando o pinguim "02" sem emparelhamento. O mesmo acontece para o pinguim "03", logo $nsp = 2$. Assim, $highestPairings=3$.

```python
line1 = "() () () () () () () () ()\n"
line2 = "## 02 00 .. .. .. .. 01 ##\n"
line3 = "## .. .. .. .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. 03 ##\n"
line7 = "## .. .. .. .. .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid2)
print(p.highestPairings(Node(p.initial)))

out: 3
```

#### Exemplo 2:

Vamos ver mais um exemplo com 8 pinguins.
```
() () () () () () () () () 
## .. 00 .. .. .. .. .. ## 
## .. 03 05 .. .. .. .. ## 
## .. .. .. .. .. .. .. ## 
## .. () .. () .. .. .. ## 
## 04 .. 06 01 .. .. .. ## 
## .. .. .. 02 .. .. .. ## 
## .. 07 .. .. .. .. .. ## 
() () () () () () () () ()
```

Novamente vamos construir uma tabela que nos permita facilmente identificar a ordem para emparelharmos os pinguins.

| Pinguim | Emparelhamento direto |
| :------ | :-------------------- |
| 00      | 03                    |
| 01      | 02 06                 |
| 02      | 01                    |
| 03      | 00 05                 |
| 04      | 06                    |
| 05      | 03 06                 |
| 06      | 01 04 05              |
| 07      |                       |

Olhando para a tabela, a ordem de emparelhamento será 
```
"06", "01", "03", "05", "00", "02", "04", "07"
```
* Comecemos com o **pinguim "06"**, este pode-se emparelhar com os pinguins "01" (com 2 emparelhamentos), "04" (com 1 emparelhamento) e "05" (com 2 emparelhamentos). Obedecendo à ordem de maior emparelhamento, então o pinguim "06" emparelha-se com os pinguins pela seguinte ordem: "01", "05" e "04". Logo o pinguim "06" emparelha-se com o "01". &rarr; **1 par**
* O **pinguim "01"** acabou de se emparelhar ao pinguim "06".
* O seguinte da lista de emparelhamentos é o **pinguim "03"**, que se pode emparelhar com os pinguins "00" (com 1 emparelhamento) e "05" (com 2 emparelhamentos). Ordenando estes pinguins pela regra de maior emparelhamentos, então o pinguim "03" forma par com o pinguim "05". &rarr; **1 par**
* O **pinguim "05"** acabou de formar par com o pinguim "03".
* De seguida, vamos tentar emparelhar o **pinguim "00"**, mas este apenas emparelha com o pinguim "03" que já formou par.
* O **pinguim "02"** não pode emparelhar com o pinguim "01".
* O **pinguim "04"** apenas poderia formar par com o pinguim "06", mas este foi emparelhado com o pinguim "01".
* Finalmente, o **pinguim "07"** não tem emparelhamento com ninguém.

Ora, temos um total de 2 pares formados ("06" com "01" e "03" com "05"), logo $np=2$ e sobram 4 pinguins sem emparelhamento, logo $nsp=4$. Valor heurístico é de 6.

```python
line1 = "() () () () () () () () ()\n"
line2 = "## .. 00 .. .. .. .. .. ##\n"
line3 = "## .. 03 05 .. .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. () .. () .. .. .. ##\n"
line6 = "## 04 .. 06 01 .. .. .. ##\n"
line7 = "## .. .. .. 02 .. .. .. ##\n"
line8 = "## .. 07 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid4 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid4)
print(p.highestPairings(Node(p.initial)))

out: 6
```

Se aplicarmos o A* em grafo a este exemplo, teremos 5 estados expandidos e uma solução com custo 5.
```python
res,expanded=astar_search_plus_count(p,p.highestPairings)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)
```
O código irá produzir o seguinte output:
```
[('05', 'S'), ('01', 'S'), ('07', 'O'), ('04', 'S'), ('00', 'S')]
Cost: 5
expanded: 5
```

#### Exemplo 3:

Vamos ver outro exemplo com mais pinguins, neste caso 10 pinguins:
```
() () () () () () () () () 
## .. .. 07 .. 01 .. .. ## 
## .. .. .. .. 05 .. .. ## 
## .. .. .. 09 .. .. .. ## 
## .. .. () () () 02 08 ## 
## .. .. .. .. 04 .. .. ## 
## .. 06 03 .. .. .. .. ## 
## .. 00 .. .. .. .. .. ## 
() () () () () () () () ()
```

Novamente vamos construir uma tabela que nos permita facilmente identificar a ordem para emparelharmos os pinguins.

| Pinguim | Emparelhamento direto |
| :------ | :-------------------- |
| 00      | 06                    |
| 01      | 05 07                 |
| 02      | 08                    |
| 03      | 06                    |
| 04      |                       |
| 05      | 01                    |
| 06      | 00 03                 |
| 07      | 01                    |
| 08      | 02                    |
| 09      |                       |

Olhando para a tabela, a ordem de emparelhamento será 
```
"01", "06", "00", "02", "03", "05", "07", "08", "04", "09"
```
* Comecemos com o **pinguim "01"**, este pode-se emparelhar com os pinguins "05" e "07", ambos apenas com um emparelhamento, logo o pinguim "01" emparelha-se com o "05". &rarr; **1 par**
* De seguida, vamos emparelhar o **pinguim "06"**, que se pode emparelhar com os pinguins "00" e "03", ambos apenas com um emparelhamento e portanto, "o pinguim "06" liga-se ao pinguim "00" (ordem do ID). &rarr; **1 par**
* O seguinte da lista de emparelhamentos é o **pinguim "00"**, mas este acabou de ser emparelhado e portanto seguimos para o seguinte.
* O **pinguim "02"** tem apenas um emparelhamento com o pinguim "08", logo formam um par. &rarr; **1 par**
* De seguida, vamos tentar emparelhar o **pinguim "03"**, mas este apenas emparelha com o pinguim "06" que já formou par.
* O **pinguim "05"** já foi emparelhado com o pinguim "01".
* O **pinguim "07"** apenas poderia formar par com o pinguim "01", mas este foi emparelhado com o pinguim "05".
* De seguida, temos o **pinguim "08"** que já foi emparelhado anteriormente com o pinguim "02".
* Finalmente, os **pinguins "04" e "09"** não têm emparelhamento com ninguém.

Ora, temos um total de 3 pares formados ("01" com "05", "06" com "00" e "02" com "08"), logo $np=3$ e sobram 4 pinguins sem emparelhamento, logo $nsp=4$. Valor heurístico é de 7.

```python
line1 = "() () () () () () () () ()\n"
line2 = "## .. .. 07 .. 01 .. .. ##\n"
line3 = "## .. .. .. .. 05 .. .. ##\n"
line4 = "## .. .. .. 09 .. .. .. ##\n"
line5 = "## .. .. () () () 02 08 ##\n"
line6 = "## .. .. .. .. 04 .. .. ##\n"
line7 = "## .. 06 03 .. .. .. .. ##\n"
line8 = "## .. 00 .. .. .. .. .. ##\n"
line9 = "() () () () () () () () ()\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid3)
print(p.highestPairings(Node(p.initial)))

out: 7
```

Se aplicarmos o A* em grafo a este exemplo, teremos 577 estados expandidos, menos estados expandidos que a heurística `Npairings` (673 estados) e que a heurística `halfPenguins` (6007 estados), e uma solução com custo 9. Se aplicarmos uma procura em largura-primeiro são expandidos 30662 estados, mostrando que o A* em grafo com uma heurística adequada torna a procura muito mais eficiente. 
```python
res,expanded=astar_search_plus_count(p,p.highestPairings)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print('No solution!')
print('expanded:',expanded)
```
O código irá produzir o seguinte output:
```
[('04', 'E'), ('03', 'E'), ('03', 'N'), ('02', 'E'), ('09', 'O'), ('05', 'O'), ('05', 'S'), ('01', 'O'), ('00', 'N')]
Cost: 9
expanded: 577
```

Se quiserem experimentar a procura em largura para este mapa, o código que conta o número de estados expandidos encontra-se a seguir, mas atenção que demora algum tempo a correr.

In [ ]:
def breadth_first_search_count(problem):
    """[Figure 3.11]"""
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node,0
    frontier = FIFOQueue()
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child,len(explored)
                frontier.append(child)
    return None,len(explored)

In [ ]:
p = PenguinsPairs(grid3)
res,expanded=breadth_first_search_count(p)
if res:
    print(res.solution())
    print('Cost:', res.path_cost)
else:
    print("Sem solução!")
print('expanded:',expanded)